In [ ]:
import os, json, csv, datetime, time
from pathlib import Path
from dotenv import load_dotenv

# --- Load env (local runs only; Actions will use repo secrets) ---
ENV_LOCAL = Path("C:/Users/kosis/Downloads/Automation/spending-dashboard/scripts/.env")
if ENV_LOCAL.exists():
    load_dotenv(ENV_LOCAL)

# --- Required env vars ---
PLAID_ENV = os.getenv("PLAID_ENV", "production").lower()
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET = os.getenv("PLAID_SECRET")
ACCESS_TOKENS_RAW = os.getenv("PLAID_ACCESS_TOKENS")  # JSON string mapping bank->access_token

if not (PLAID_CLIENT_ID and PLAID_SECRET and ACCESS_TOKENS_RAW):
    raise SystemExit("Missing PLAID_CLIENT_ID / PLAID_SECRET / PLAID_ACCESS_TOKENS env vars")

try:
    ACCESS_TOKENS = json.loads(ACCESS_TOKENS_RAW)
    assert isinstance(ACCESS_TOKENS, dict) and ACCESS_TOKENS
except Exception as e:
    raise SystemExit(f"PLAID_ACCESS_TOKENS must be JSON object, e.g. {{\"Bank1\":\"access-...\"}}. Error: {e}")

# --- Plaid client (ONLY production or sandbox) ---
from plaid import ApiClient, Configuration
from plaid.api.plaid_api import PlaidApi
from plaid.model.country_code import CountryCode
from plaid.model.transactions_sync_request import TransactionsSyncRequest
from plaid.model.transactions_get_request import TransactionsGetRequest
from plaid.model.account_subtype import AccountSubtype
from plaid.model.transactions_get_request_options import TransactionsGetRequestOptions

BASE_URLS = {
    "production": "https://production.plaid.com",
    "sandbox": "https://sandbox.plaid.com",
}
if PLAID_ENV not in BASE_URLS:
    raise SystemExit(f"PLAID_ENV must be 'production' or 'sandbox' (got '{PLAID_ENV}')")
BASE_URL = BASE_URLS[PLAID_ENV]

config = Configuration(host=BASE_URL, api_key={"clientId": PLAID_CLIENT_ID, "secret": PLAID_SECRET})
client = PlaidApi(ApiClient(config))

# --- Folders & state ---
ROOT = Path(".")
RAW_DIR = ROOT / "data" / "raw"
STATE_DIR = ROOT / ".state"
STATE_DIR.mkdir(parents=True, exist_ok=True)
RAW_DIR.mkdir(parents=True, exist_ok=True)

CURSOR_PATH = STATE_DIR / "plaid_cursors.json"
if CURSOR_PATH.exists():
    cursors = json.loads(CURSOR_PATH.read_text())
else:
    cursors = {}  # {bank_name: cursor}

today = datetime.date.today().isoformat()

def write_csv(bank: str, rows: list):
    if not rows:
        return None
    cols = [
        "account_id","transaction_id","authorized_date","date","name","merchant_name",
        "amount","iso_currency_code","pending","payment_channel","category","category_id"
    ]
    out_path = RAW_DIR / f"{today}_{bank}.csv"
    with out_path.open("w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=cols)
        w.writeheader()
        for t in rows:
            w.writerow({
                "account_id": t.get("account_id"),
                "transaction_id": t.get("transaction_id"),
                "authorized_date": t.get("authorized_date"),
                "date": t.get("date"),
                "name": t.get("name"),
                "merchant_name": t.get("merchant_name"),
                "amount": t.get("amount"),
                "iso_currency_code": t.get("iso_currency_code"),
                "pending": t.get("pending"),
                "payment_channel": t.get("payment_channel"),
                "category": "|".join(t.get("category") or []),
                "category_id": t.get("category_id"),
            })
    return out_path

def fetch_sync(access_token: str, cursor: str | None):
    """Use transactions/sync to get new/modified transactions since last cursor.
       On first run (cursor is None), Plaid returns up to ~30 days by default."""
    added, modified, removed = [], [], []
    has_more = True
    next_cursor = cursor
    while has_more:
        req = TransactionsSyncRequest(access_token=access_token, cursor=next_cursor)
        res = client.transactions_sync(req).to_dict()
        added.extend(res.get("added", []))
        modified.extend(res.get("modified", []))
        removed.extend(res.get("removed", []))
        has_more = bool(res.get("has_more"))
        next_cursor = res.get("next_cursor")
        time.sleep(0.2)
    return added, modified, removed, next_cursor

def initial_seed_last_90_days(access_token: str):
    """Optional helper to seed more history on first run; comment out if not needed."""
    end_date = datetime.date.today()
    start_date = end_date - datetime.timedelta(days=89)
    req = TransactionsGetRequest(
        access_token=access_token,
        start_date=start_date.isoformat(),
        end_date=end_date.isoformat(),
        options=TransactionsGetRequestOptions(count=500)
    )
    res = client.transactions_get(req).to_dict()
    txs = res.get("transactions", [])
    total = res.get("total_transactions", len(txs))
    while len(txs) < total:
        req.options.offset = len(txs)
        res = client.transactions_get(req).to_dict()
        txs.extend(res.get("transactions", []))
        time.sleep(0.2)
    return txs

any_changes = False
for bank, token in ACCESS_TOKENS.items():
    print(f"\n=== {bank} ===")
    cursor = cursors.get(bank)
    if cursor:
        print("Using existing cursor…")
        added, modified, removed, next_cursor = fetch_sync(token, cursor)
        changed = added + modified
    else:
        print("No cursor yet. Doing initial sync (last ~30 days via /sync).")
        added, modified, removed, next_cursor = fetch_sync(token, None)
        # Optionally seed more history:
        # print("Fetching last 90d via /transactions/get for seed…")
        # added = initial_seed_last_90_days(token)

    print(f"Added: {len(added)}, Modified: {len(modified)}, Removed: {len(removed)}")
    out_path = write_csv(bank, added + modified)
    if out_path:
        print(f"Wrote: {out_path}")
        any_changes = True
    else:
        print("No new/modified transactions to write.")

    if next_cursor and next_cursor != cursor:
        cursors[bank] = next_cursor

CURSOR_PATH.write_text(json.dumps(cursors, indent=2), encoding="utf-8")
print(f"\nSaved cursors -> {CURSOR_PATH}")

if not any_changes:
    print("No CSV changes this run.")
